In [2]:
import torch

In [4]:
def load_file(path):
  with open(path) as f:
    return f.read().strip()

In [7]:
def parse_line(line):
  return [int(c) for c in line]

def parse_file(content):
  lines = content.split('\n')   
  trees = [parse_line(line) for line in lines]
  return torch.tensor(trees)

In [12]:
x = parse_file(load_file('example.txt'))
x

tensor([[3, 0, 3, 7, 3],
        [2, 5, 5, 1, 2],
        [6, 5, 3, 3, 2],
        [3, 3, 5, 4, 9],
        [3, 5, 3, 9, 0]])

In [16]:
x.roll(1, 1)
x[:,0] = 0
x

tensor([[0, 0, 3, 7, 3],
        [0, 5, 5, 1, 2],
        [0, 5, 3, 3, 2],
        [0, 3, 5, 4, 9],
        [0, 5, 3, 9, 0]])

In [52]:
def get_visible(Trees):
  Trees = Trees.clone()
  VisibleTrees = torch.zeros(Trees.shape)

  for col in range(Trees.shape[0]):
    for row in range(Trees.shape[1]):
      tree = Trees[row, col]
      leftTrees = Trees[row, :col] >= tree
      rightTrees = Trees[row, col+1:] >= tree
      downTrees = Trees[row+1:, col] >= tree
      upTrees = Trees[:row, col] >= tree
      isVisible = leftTrees.sum() == 0 or rightTrees.sum() == 0 or downTrees.sum() == 0 or upTrees.sum() == 0
      VisibleTrees[row, col] = isVisible

  return VisibleTrees

In [53]:
X = parse_file(load_file("input.txt"))
print(X)
V = get_visible(X)
print(V)
V.sum()

tensor([[2, 2, 2,  ..., 0, 4, 0],
        [1, 1, 0,  ..., 0, 0, 1],
        [1, 3, 0,  ..., 0, 3, 3],
        ...,
        [0, 3, 0,  ..., 0, 0, 2],
        [4, 0, 4,  ..., 4, 3, 0],
        [3, 2, 4,  ..., 1, 3, 2]])
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        [1., 1., 0.,  ..., 0., 0., 1.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


tensor(1533.)

In [41]:
print(X[1,1])
X[1, 2:] > X[1, 1]

tensor(5)


tensor([False, False, False])

In [90]:
def get_score(tree, v):
  blocking_trees = v >= tree
  if blocking_trees.sum() == 0:
    return len(blocking_trees)
  return torch.nonzero(v >= tree)[0][0] + 1

def get_scenic_score(Trees):
  Trees = Trees.clone()
  TreeScores = torch.zeros(Trees.shape)
  for col in range(Trees.shape[0]):
    for row in range(Trees.shape[1]):
      tree = Trees[row, col]
      leftTrees = Trees[row, :col].flip(0)
      rightTrees = Trees[row, col+1:]
      downTrees = Trees[row+1:, col]
      upTrees = Trees[:row, col].flip(0)

      score = get_score(tree, rightTrees) * get_score(tree, leftTrees) * get_score(tree, downTrees) * get_score(tree, upTrees)
      TreeScores[row, col] = score
      
  return TreeScores

In [92]:
X = parse_file(load_file("example.txt"))
Scores = get_scenic_score(X)
print(Scores)
Scores.max()

tensor(8.)